# Convert COVID-19 data from CSSE to Gapminder format

load csv  data soruces: https://github.com/CSSEGISandData/COVID-19
CC-BY: Masaki Yamabe (masakiyamabe.com)

In [1]:
#Library
import numpy as np
import pandas as pd

source_confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"
source_death = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"
source_recoverd = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv"

df_confirmed = pd.read_csv(source_confirmed)
df_death = pd.read_csv(source_death)
df_recoverd  = pd.read_csv(source_recoverd)

In [2]:
#delete unuse column
del df_confirmed['Province/State']
del df_confirmed['Lat']
del df_confirmed['Long']

del df_death['Province/State']
del df_death['Lat']
del df_death['Long']

del df_recoverd['Province/State']
del df_recoverd['Lat']
del df_recoverd['Long']
df_recoverd.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20
0,Thailand,0,0,0,0,2,2,5,5,5,...,34,35,35,35,35,41,42,42,42,42
1,Japan,0,0,0,0,1,1,1,1,1,...,118,118,118,118,144,144,144,150,191,232
2,Singapore,0,0,0,0,0,0,0,0,0,...,96,97,105,105,109,114,114,114,124,140
3,Nepal,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
4,Malaysia,0,0,0,0,0,0,0,0,0,...,26,26,35,42,42,49,60,75,87,114


In [3]:
# group by "country/region"
df_confirmed = df_confirmed.groupby("Country/Region").apply(lambda x: x.sum()).drop('Country/Region', axis=1).reset_index()
df_death =  df_death.groupby("Country/Region").apply(lambda x: x.sum()).drop('Country/Region', axis=1).reset_index()
df_recoverd =  df_recoverd.groupby("Country/Region").apply(lambda x: x.sum()).drop('Country/Region', axis=1).reset_index()

In [4]:
#insert case column

df_confirmed.insert(1,"case",'confirmed')
df_death.insert(1,"case",'deaths')
df_recoverd.insert(1,"case",'recoverd')

In [5]:
df_combined = pd.concat([df_confirmed, df_death, df_recoverd])
df_combined.head()

,Country/Region,case,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20
0,Afghanistan,confirmed,0,0,0,0,0,0,0,0,...,7,7,11,16,21,22,22,22,24,24
1,Albania,confirmed,0,0,0,0,0,0,0,0,...,23,33,38,42,51,55,59,64,70,76
2,Algeria,confirmed,0,0,0,0,0,0,0,0,...,24,26,37,48,54,60,74,87,90,139
3,Andorra,confirmed,0,0,0,0,0,0,0,0,...,1,1,1,1,2,39,39,53,75,88
4,Angola,confirmed,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2


In [6]:
#convert date format
from datetime import datetime as dt
cols = np.array(df_combined.columns)


for i,word in enumerate(cols):
    if i>1:
        s = cols[i]
        tdatetime = dt.strptime(s, '%m/%d/%y')
        cols[i] = tdatetime.strftime('%Y%m%d')
        
# set new columns to df
df_combined.columns = cols
df_combined.head()

,Country/Region,case,20200122,20200123,20200124,20200125,20200126,20200127,20200128,20200129,...,20200312,20200313,20200314,20200315,20200316,20200317,20200318,20200319,20200320,20200321
0,Afghanistan,confirmed,0,0,0,0,0,0,0,0,...,7,7,11,16,21,22,22,22,24,24
1,Albania,confirmed,0,0,0,0,0,0,0,0,...,23,33,38,42,51,55,59,64,70,76
2,Algeria,confirmed,0,0,0,0,0,0,0,0,...,24,26,37,48,54,60,74,87,90,139
3,Andorra,confirmed,0,0,0,0,0,0,0,0,...,1,1,1,1,2,39,39,53,75,88
4,Angola,confirmed,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2


In [7]:
df_combined.describe()

,20200122,20200123,20200124,20200125,20200126,20200127,20200128,20200129,20200130,20200131,...,20200312,20200313,20200314,20200315,20200316,20200317,20200318,20200319,20200320,20200321
count,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,...,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000,498.000000
mean,1.204819,1.407631,2.014056,3.042169,4.469880,6.164659,11.678715,12.901606,17.164659,20.807229,...,404.391566,443.469880,470.957831,501.847390,535.624498,574.070281,616.164659,677.568273,744.419679,821.277108
std,24.595934,28.851394,41.268003,63.048225,93.030901,128.984008,246.949308,272.854145,364.902370,439.397954,...,4658.546259,4740.641491,4822.448731,4910.440832,4993.548454,5089.131470,5211.159032,5387.573265,5602.016440,5860.766110
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,6.000000,9.000000,10.000000,13.000000,16.750000,21.750000,28.000000,32.000000,42.750000
max,548.000000,643.000000,920.000000,1406.000000,2075.000000,2877.000000,5509.000000,6087.000000,8141.000000,9802.000000,...,80932.000000,80945.000000,80977.000000,81003.000000,81033.000000,81058.000000,81102.000000,81156.000000,81250.000000,81305.000000


In [8]:
df_combined.to_csv("covid19_gapminder.csv",index=False)